In [ ]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 14.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requi

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from gensim.models import Word2Vec
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

## Read the data

In [ ]:
data=pd.read_excel('/content/IMDB Dataset_sample.xlsx')

In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


## Split the data

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(data['review'],
                                               data['sentiment'],
                                               test_size=0.2,
                                               random_state=42)

## We are individually importing the NLP packages

In [ ]:
data.shape,X_train.shape,y_train.shape

((1114, 2), (891,), (891,))

In [ ]:
X_train.shape,y_train.shape

((891,), (891,))

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

## Creating a preprocess function

- lower the text

- remove the punctuations

- tokenize the text

- apply the stop words

In [ ]:
stop_words=set(stopwords.words('english'))
def preprocess(text):
  text=text.lower()
  text="".join([word for word in text if word not in string.punctuation])
  tokens=word_tokenize(text)
  tokens=[word for word in tokens if word not in stop_words]
  return " ".join(tokens)
X_train=X_train.apply(preprocess)
X_test=X_test.apply(preprocess)

In [ ]:

X_train[0]

'one reviewers mentioned watching 1 oz episode youll hooked right exactly happened mebr br first thing struck oz brutality unflinching scenes violence set right word go trust show faint hearted timid show pulls punches regards drugs sex violence hardcore classic use wordbr br called oz nickname given oswald maximum security state penitentary focuses mainly emerald city experimental section prison cells glass fronts face inwards privacy high agenda em city home manyaryans muslims gangstas latinos christians italians irish moreso scuffles death stares dodgy dealings shady agreements never far awaybr br would say main appeal show due fact goes shows wouldnt dare forget pretty pictures painted mainstream audiences forget charm forget romanceoz doesnt mess around first episode ever saw struck nasty surreal couldnt say ready watched developed taste oz got accustomed high levels graphic violence violence injustice crooked guards wholl sold nickel inmates wholl kill order get away well mannere

In [ ]:
len(X_train)

891

## Tokenization

In [ ]:
# Train the Word2Vec model
#we are calling each sentence from X_train
# we are spliting: it beacouse words
# we have 891 lists are there
# each list has words
sentences=[sentence.split() for sentence in X_train]
len(sentences)

891

In [ ]:
sentences[1]

['take',
 'cast',
 'experienced',
 'wellknown',
 'actors',
 'put',
 'together',
 'stupid',
 'movie',
 'nimrod',
 'antel',
 'answer',
 'armored',
 'six',
 'coworkers',
 'armored',
 'car',
 'business',
 'decide',
 'steal',
 'large',
 'shipment',
 'cash',
 'get',
 'first',
 'base',
 'plans',
 'everything',
 'unravels',
 'quickly',
 'plot',
 'like',
 'youd',
 'think',
 'couldnt',
 'bad',
 'least',
 'action',
 'movie',
 'however',
 'first',
 '40',
 'minutes',
 'movie',
 'see',
 'appear',
 '6',
 'normal',
 'everyday',
 'kind',
 'guys',
 'joke',
 'laugh',
 'drinks',
 'together',
 'etc',
 'suddenly',
 'learn',
 'theyre',
 'planning',
 'rob',
 'business',
 'hero',
 'ty',
 'columbus',
 'short',
 'sucked',
 'scheme',
 'cold',
 'cruel',
 'world',
 'even',
 'though',
 'hes',
 'decorated',
 'veteran',
 'nice',
 'guy',
 'reliable',
 'employee',
 'oh',
 'oh',
 'last',
 '40',
 'minutes',
 'film',
 'former',
 'regular',
 'guys',
 'nearly',
 'turn',
 'moneycrazed',
 'psychos',
 'willing',
 'butcher',
 'c

## Fine tune the word 2 vec model

**size** - size of the vectors we want

**window** - number words before and after the focus word that it donsider as context for the word

**min_count** - the number od times a word must appear in our corpus in order to create a word vector

In [ ]:
w2v_model=Word2Vec(sentences,window=5,min_count=5,workers=4)
w2v_model
# default vector size=100


In [ ]:
len(w2v_model.wv['king'])

100

In [ ]:
w2v_model.corpus_total_words

108286

In [ ]:
w2v_model.wv.most_similar("king")

[('shot', 0.999565064907074),
 ('may', 0.9995535016059875),
 ('another', 0.9995452165603638),
 ('long', 0.9995435476303101),
 ('often', 0.9995416402816772),
 ('trying', 0.9995408654212952),
 ('father', 0.999529242515564),
 ('young', 0.9995285272598267),
 ('yet', 0.9995224475860596),
 ('also', 0.9995220899581909)]

In [ ]:
from numpy.linalg import norm

In [ ]:
similarity=w2v_model.wv.similarity("king",'john')
similarity

0.9993688

In [ ]:
from math import cos
vector1=w2v_model.wv['king']
vector2=w2v_model.wv['john']

cos_similarity=np.dot(vector1,vector2)/(np.linalg.norm(vector1)*np.linalg.norm(vector2))
print(cos_similarity)

0.99936885


In [ ]:
def vectorize(sentence):
  words=sentence.split()
  words_vecs=[w2v_model.wv[word] for word in words if word in w2v_model.wv]
  if len(words_vecs)==0:
    return np.zeros(100)
  words_vecs=np.array(words_vecs)
  return words_vecs.mean(axis=0)

In [ ]:
X_train=np.array([vectorize(sentence) for sentence in X_train])
X_test=np.array([vectorize(sentence) for sentence in X_test])

##Develop the ML model

In [ ]:
# Train a classification model
clf=LogisticRegression()
clf.fit(X_train,y_train)
#Evaluate the model
y_pred=clf.predict(X_test)
print("Accuracy:",accuracy_score(y_test,y_pred))
print("Precission:",precision_score(y_test,y_pred,pos_label='positive'))
print('Recall:',recall_score(y_test,y_pred,pos_label='positive'))
print('F1 score:',f1_score(y_test,y_pred,pos_label='positive'))


Accuracy: 0.5515695067264574
Precission: 0.5576923076923077
Recall: 0.5178571428571429
F1 score: 0.5370370370370371


In [ ]:
X_test='good movie'
X_test=preprocess(X_test)
X_test=np.array([vectorize(X_test)])
print(len(X_test))
clf.predict(X_test)

1


array(['negative'], dtype=object)

In [ ]:
X_test='The movie was worth time and good experience'
X_test=preprocess(X_test)
X_test=np.array([vectorize(X_test)])
print(len(X_test))
clf.predict(X_test)

1


array(['negative'], dtype=object)

In [ ]:
X_test='bad movie waste movie'
X_test=preprocess(X_test)
X_test=np.array([vectorize(X_test)])
print(len(X_test))
clf.predict(X_test)

1


array(['negative'], dtype=object)